<a href="https://colab.research.google.com/github/GulnazaS/Deploy_model/blob/main/ML_deploy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip install langchain-huggingface
!pip install sentencepiece accelerate
!pip install -U bitsandbytes
!pip install peft
!pip install llama-index-readers-wikipedia wikipedia

In [ ]:
!pip install pyvis

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch
from pyvis.network import Network
from huggingface_hub import login
HF_TOKEN="HFL"
# Вставьте ваш токен (здесь указан временный токен)
login(HF_TOKEN, add_to_git_credential=True)

In [ ]:
!pip install trl

In [ ]:
!pip install gradio

In [ ]:
  print(docs)

In [ ]:
import gradio as gr
# Загрузка страниц из википедии
from llama_index.readers.wikipedia import WikipediaReader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import re

# Загружаем модель для исправления орфографических ошибок
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sage-fredt5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("ai-forever/sage-fredt5-large", device_map='auto')

def process_documents(docs):
    def clean_text(text):
        # Удаление HTML-тегов, если они есть
        text = re.sub(r'<.*?>', '', text)
        # Удаление всех символов, кроме букв и пробелов
        text = re.sub(r'[^а-яА-ЯёЁ\s]', '', text)
        # Удаление лишних пробелов
        text = re.sub(r'\s+', ' ', text).strip()
        return text

    # Применение функции очистки к каждому документу
    cleaned_docs = [clean_text(doc.text) for doc in docs]

    # Возвращаем очищенный текст
    return "\n\n".join(cleaned_docs)

# Инициализация объекта WikipediaReader
reader = WikipediaReader()

def correct_text(input_text):
    inputs = tokenizer(input_text, max_length=512, padding="longest", truncation=True, return_tensors="pt")
    outputs = model.generate(**inputs.to(model.device), max_length=inputs["input_ids"].size(1) * 1.5)
    corrected_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return corrected_text[0]  # Возвращаем первый элемент, так как batch_decode возвращает список

def load_wikipedia_data():
    # Загрузка данных из Википедии
    docs = reader.load_data(
        pages=["История искусства"],  # запрос раздела на тему ИИ
        lang_prefix='ru'
    )
    return docs

with gr.Blocks() as app:
    gr.Markdown("# Исправление орфографических ошибок и парсинг текста")
    with gr.Row():
        output_text = gr.Textbox(lines=5, placeholder="Данные из Википедии будут здесь")
        parse_text = gr.Textbox(lines=5, placeholder="Распарсенный текст будет здесь")
    with gr.Row():
        load_button = gr.Button("Загрузить данные из Википедии")
        parse_button = gr.Button("Распарсить текст")
    with gr.Row():
       corrected_textbox = gr.Textbox(lines=5, placeholder="Исправленный текст будет отображаться здесь")
    with gr.Row():
        correct_button = gr.Button("Исправить текст")

    load_button.click(fn=load_wikipedia_data, outputs=output_text)
    parse_button.click(fn=lambda: process_documents(load_wikipedia_data()), outputs=parse_text)
    correct_button.click(fn=correct_text, inputs=parse_text, outputs=corrected_textbox)

app.launch()